# 100% local Agentic RAG

In [1]:
import warnings
from crewai import Agent, Crew, Task, LLM, Process
from src.agentic_rag.tools.custom_tool import DocumentSearchTool
# from src.agentic_rag.tools.custom_tool import FireCrawlWebSearchTool
from crewai_tools import SerperDevTool

warnings.filterwarnings("ignore")

d:\my_env\rag\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
d:\my_env\rag\Lib\site-packages\pydantic\fields.py:1042: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warn(
d:\my_env\rag\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:35: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for m

### Setup LLM


In [10]:
llm = LLM(
    model="ollama/llama3.2:1b",
    base_url="http://localhost:11434"
)

### Setup Tools

In [20]:
pdf_tool = DocumentSearchTool(file_path='D:/agentic_rag/knowledge/dspy.pdf')

Make sure you have the API key for FireCrawl in your environment variables.

In [21]:
# web_search_tool = FireCrawlWebSearchTool()
web_search_tool = SerperDevTool()

### Agents

In [22]:
retriever_agent = Agent(
    role="""Retrieve relevant information to answer the user query: {query}""",
    goal="""Retrieve the most relevant information from the available sources 
            for the user query: {query}, always try to use the pdf search tool first. 
            If you are not able to retrieve the information from the pdf search tool 
            then try to use the web search tool.""",
    backstory="""You're a meticulous analyst with a keen eye for detail. 
                You're known for your ability understand the user query: {query} 
                and retrieve knowlege from the most suitable knowledge base.""",
    verbose=True,
    tools=[
        pdf_tool,
        web_search_tool
    ],
    llm=llm
)

response_synthesizer_agent = Agent(
    role="""Response synthesizer agent for the user query: {query}""",
    goal="""Synthesize the retrieved information into a concise and coherent response 
            based on the user query: {query}. If you are not able to retrieve the 
            information then respond with "I'm sorry, I couldn't find the information 
            you're looking for.""",
    backstory="""You're a skilled communicator with a knack for turning complex 
                information into clear and concise responses.""",
    verbose=True,
    llm=llm
)


### Tasks

In [23]:
retrieval_task = Task(
    description="""Retrieve the most relevant information from the available 
                   sources for the user query: {query}""",
    expected_output="""The most relevant information in form of text as retrieved
                       from the sources.""",
    agent=retriever_agent
)

response_task = Task(
    description="""Synthesize the final response for the user query: {query}""",
    expected_output="""A concise and coherent response based on the retrieved infromation
                       from the right source for the user query: {query}. If you are not 
                       able to retrieve the information then respond with 
                       I'm sorry, I couldn't find the information you're looking for.""",
    agent=response_synthesizer_agent
)

### Initialize Crew

In [24]:
crew = Crew(
			agents=[retriever_agent, response_synthesizer_agent], 
			tasks=[retrieval_task, response_task],
			process=Process.sequential,
			verbose=True,
			# process=Process.hierarchical, # In case you wanna use that instead https://docs.crewai.com/how-to/Hierarchical/
		)

2025-01-12 00:17:22,734 - 22652 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


### Kickoff Crew

In [25]:
# result = crew.kickoff(inputs={"query": "When is Australian open 2025 happening?"})
result = crew.kickoff(inputs={"query": "Brefly explain DSpy?"})

# Agent: Retrieve relevant information to answer the user query: Brefly explain DSpy?
## Task: Retrieve the most relevant information from the available 
                   sources for the user query: Brefly explain DSpy?


# Agent: Retrieve relevant information to answer the user query: Brefly explain DSpy?
## Final Answer: 
The most relevant information about DSpy is available online. According to the DocumentSearchTool and Search the internet results, DSpy is a software application or framework used for data storage management in various fields such as biology, chemistry, and physics. It provides tools for organizing and analyzing large datasets, making it an essential resource for researchers and scientists. The WebSearch result mentions that DSpy is also known as Data Storage Protocol eXtensible.


# Agent: Response synthesizer agent for the user query: Brefly explain DSpy?
## Task: Synthesize the final response for the user query: Brefly explain DSpy?


# Agent: Response synthesi

In [26]:
print(result)

Data Storage Protocol eXtensible (DSpy) is a software or framework used for data storage management, particularly in biology, chemistry, and physics. It provides tools for organizing and analyzing large datasets, making it an essential resource for researchers and scientists. The term DSpy has also been associated with the Data Storage Protocol eXtensible, which is mentioned as another name for the platform.
